# Creación de un mapa de contaminación

Se pretende cargar la posición de las estaciones de medición de contaminantes junto con sus contaminantes y realizar una interpolación de estos contaminantes en todo un mallado entre las estaciones conocidas.

In [1]:
import numpy as np
import pandas as pd
import re
from scipy.interpolate import SmoothBivariateSpline,interp2d,griddata
import matplotlib.pyplot as plt 
import pyarrow #uninstall libboost & conda install pyarrow=0.13.* -c conda-forge

In [2]:
def dms2dd(input):
    "Esta función tranforma un punto expresado en coordenadas geográficas a coordenadas UTM"
    degrees,minutes,seconds,direction=re.split('[º\´"]+',str(input))
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(3600);
    if direction == 'S' or direction == 'O':
        dd *= -1
    return dd;

Se leeran las coordenadas de las estaciones y se filtrarán las que son de nuestro interés

In [3]:
posicion_estaciones=pd.read_excel('./data/EstacionesLocalizacion.xlsx') #Se carga el fichero

Se filtra el fichero de estaciones a la estaciones de nuestro estudio

In [4]:
Estacion=[4,8,11,16,18,35,36,38,39,40,47,48,49,50,56]
#[4,8,11,35,38,39,47,48,49,50],
posicion_estaciones=posicion_estaciones[posicion_estaciones['Estacion'].isin(Estacion)]
posicion_estaciones.set_index('Estacion',inplace=True)
posicion_estaciones

,Columna1,Latitud,Longitud
Estacion,,,
4,Plaza de España,"40º25´25.98""N","3º42´43.91""O"
8,Escuelas Aguirre,"40º25´17.63""N","3º40´56.22""O"
11,Av Ramon y Cajal,"40º27´5.29""N","3º40´38.5""O"
16,Arturo Soria,"40º26´24.20""N","3º38´21.17""O"
18,Calle Farolillo,"40º23´41.22""N","3º43´54.61""O"
35,Plaza del carmen,"40º25´9.15""N","3º42´11.4""O"
36,Moratalaz,"40º24´28.64""N","3º38´43.02""O"
38,Cuatro Caminos,"40º26´43.97""N","3º42´25.64""O"
39,Barrio del pilar,"40º28´41.64""N","3º42´41.53""O"


Se convertirán los valores de Latitud y Longitud en coordenadas decimales

In [5]:
posicion_estaciones[['Latitud','Longitud']]=posicion_estaciones[['Latitud','Longitud']].applymap(lambda x: dms2dd(x))

In [6]:
posicion_estaciones.sort_values(['Latitud','Longitud'])

,Columna1,Latitud,Longitud
Estacion,,,
56,Plaza Fernandez Ladreda,40.385028,-3.718761
40,Vallecas,40.388147,-3.651528
18,Calle Farolillo,40.394783,-3.731836
47,Mendez Alavaro,40.398103,-3.686825
36,Moratalaz,40.407956,-3.645283
49,Retiro,40.414444,-3.682583
35,Plaza del carmen,40.419208,-3.703167
8,Escuelas Aguirre,40.421564,-3.682283
4,Plaza de España,40.423883,-3.712197


In [7]:
sortLatitud=posicion_estaciones['Latitud'].sort_values()
sortLongitud=posicion_estaciones['Longitud'].sort_values()

Se desea obtener una matriz de puntos sobre la que realizar una interpolación. La interpolación se realizará sobre los puntos conocidos, para obtener una matriz de valores contaminantes, que representará la polución en cada espacio geografico de Madrid capital

In [8]:
Latitudes=[]
Longitudes=[]
for i in range(0,(len(sortLatitud))):
    if i==(len(sortLatitud)-1):
        lastLat=np.round(sortLatitud.iloc[i],decimals=6)
        lastLon=np.round(sortLongitud.iloc[i],decimals=6)
        Latitudes.append(lastLat)
        Longitudes.append(lastLon)
    else:
        inicioLat=sortLatitud.iloc[i]
        inicioLon=sortLongitud.iloc[i]
        finLat=sortLatitud.iloc[i+1]
        finLon=sortLongitud.iloc[i+1]
        if finLat-inicioLat>0.002:
            betweenLat=np.round(np.arange(inicioLat,finLat,0.001),decimals=6)
            Latitudes.extend(betweenLat)
        if finLon-inicioLon>0.002:
            betweenLon=np.round(np.arange(inicioLon,finLon,0.001),decimals=6)
            Longitudes.extend(betweenLon)
        

In [9]:
datos_estaciones=pd.read_csv('./data/NO2_estaciones.csv')
datos_estaciones.head()

,FECHA,Estacion4,Estacion8,Estacion11,Estacion16,Estacion18,Estacion35,Estacion36,Estacion38,Estacion39,Estacion40,Estacion47,Estacion48,Estacion49,Estacion50,Estacion56
0,2015-01-01 01:00:00,83.0,168.0,93.0,104.0,72.0,109.0,107.0,102.0,104.0,134.0,112.0,108.0,136.0,127.0,115.0
1,2015-01-01 02:00:00,77.0,203.0,224.0,128.0,81.0,133.0,184.0,183.0,168.0,141.0,120.0,114.0,111.0,170.0,149.0
2,2015-01-01 03:00:00,86.0,220.0,152.0,142.0,72.0,126.0,195.0,193.0,203.0,138.0,109.0,141.0,109.0,118.0,110.0
3,2015-01-01 04:00:00,65.0,169.0,151.0,120.0,69.0,112.0,189.0,122.0,147.0,162.0,87.0,145.0,99.0,126.0,99.0
4,2015-01-01 05:00:00,53.0,154.0,106.0,117.0,66.0,109.0,225.0,97.0,159.0,104.0,76.0,142.0,103.0,140.0,96.0


In [10]:
fechas=list(datos_estaciones.FECHA)

In [11]:
fechas2015=[]
fechas2015_dia=[]
fechas2016=[]
fechas2016_dia=[]
fechas2017=[]
fechas2017_dia=[]
fechas2018=[]
fechas2018_dia=[]
fechas2019=[]
fechas2019_dia=[]
fechas2020=[]
fechas2020_dia=[]

for date in fechas:
    fechita=date[0:10]
    if re.findall('2015', date):
        fechas2015.append(date)
        if fechita in fechas2015_dia:
            pass
        else:
            fechas2015_dia.append(fechita)
            
    elif re.findall('2016', date):
        fechas2016.append(date)
        if fechita in fechas2016_dia:
            pass
        else:
            fechas2016_dia.append(fechita)
            
    elif re.findall('2017', date):
        fechas2017.append(date)
        if fechita in fechas2017_dia:
            pass
        else:
            fechas2017_dia.append(fechita)
            
    elif re.findall('2018', date):
        fechas2018.append(date)
        if fechita in fechas2018_dia:
            pass
        else:
            fechas2018_dia.append(fechita)
            
    elif re.findall('2019', date):
        fechas2019.append(date)
        if fechita in fechas2019_dia:
            pass
        else:
            fechas2019_dia.append(fechita)
    else:
        fechas2020.append(date)
        if fechita in fechas2020_dia:
            pass
        else:
            fechas2020_dia.append(fechita)

In [12]:
ix1 = pd.MultiIndex.from_product([fechas,Latitudes])
mapa = pd.DataFrame(index=ix1, columns=Longitudes, dtype=np.float16)

In [ ]:
%%time
for ano in ['2016','2017','2018','2019','2020']:
    variable='fechas'+ano
    variable_dia='fechas'+ano+'_dia'
    fechas=eval(variable)
    fechas_dia=eval(variable_dia)
    for date in fechas:
        print(date)
        x=[]
        y=[]
        Z=[]
        for i in Estacion:
            lat=posicion_estaciones.loc[i,'Latitud'].round(decimals=6)
            lon=posicion_estaciones.loc[i,'Longitud'].round(decimals=6)
            Est='Estacion'+str(i)
            sust=float(datos_estaciones[datos_estaciones['FECHA']==date][Est])
            x.append(lon)
            y.append(lat)
            Z.append(sust)
        interp_spline = SmoothBivariateSpline(y,x,Z,kx=2,ky=2)
        mapa.loc[(date),:] = interp_spline(Latitudes,Longitudes)
    
    if ano=='2016':
        mapa2016_mediadia=pd.DataFrame()
        for dia in fechas_dia:
            pattern=dia
            matching = [s for s in datos_estaciones.FECHA if pattern in s]
            fecha_inicio=matching[0]
            fecha_fin=matching[-1]
            media2016=mapa2016.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
            media2016=media2016.stack().reset_index()
            media2016['fecha']=pattern
            media2016['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
            media2016['contaminacion']=media2016['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
            media2016['contaminacion']=media2016['contaminacion'].astype(np.int32)
            media2016['fecha']=pd.to_datetime(media2016['fecha'])
            mapa2016_mediadia=mapa2016_mediadia.append(media2016)
        mapa2016_mediadia.to_csv('./data/mapa2016_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')

    elif ano=='2017':
        mapa2017_mediadia=pd.DataFrame()
        for dia in fechas_dia:
            pattern=dia
            matching = [s for s in datos_estaciones.FECHA if pattern in s]
            fecha_inicio=matching[0]
            fecha_fin=matching[-1]
            media2017=mapa2017.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
            media2017=media2017.stack().reset_index()
            media2017['fecha']=pattern
            media2017['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
            media2017['contaminacion']=media2017['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
            media2017['contaminacion']=media2017['contaminacion'].astype(np.int32)
            media2017['fecha']=pd.to_datetime(media2017['fecha'])
            mapa2017_mediadia=mapa2017_mediadia.append(media2017)
        mapa2017_mediadia.to_csv('./data/mapa2017_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')

    elif ano=='2018':
        mapa2018_mediadia=pd.DataFrame()
        for dia in fechas_dia:
            pattern=dia
            matching = [s for s in datos_estaciones.FECHA if pattern in s]
            fecha_inicio=matching[0]
            fecha_fin=matching[-1]
            media2018=mapa2018.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
            media2018=media2018.stack().reset_index()
            media2018['fecha']=pattern
            media2018['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
            media2018['contaminacion']=media2018['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
            media2018['contaminacion']=media2018['contaminacion'].astype(np.int32)
            media2018['fecha']=pd.to_datetime(media2018['fecha'])
            mapa2018_mediadia=mapa2018_mediadia.append(media2017)
        mapa2018_mediadia.to_csv('./data/mapa2018_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')

    elif ano=='2019':
        mapa2019_mediadia=pd.DataFrame()
        for dia in fechas_dia:
            pattern=dia
            matching = [s for s in datos_estaciones.FECHA if pattern in s]
            fecha_inicio=matching[0]
            fecha_fin=matching[-1]
            media2019=mapa2019.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
            media2019=media2019.stack().reset_index()
            media2019['fecha']=pattern
            media2019['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
            media2019['contaminacion']=media2019['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
            media2019['contaminacion']=media2019['contaminacion'].astype(np.int32)
            media2019['fecha']=pd.to_datetime(media2019['fecha'])
            mapa2019_mediadia=mapa2019_mediadia.append(media2019)
        mapa2019_mediadia.to_csv('./data/mapa2019_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')

    else:
        mapa2020_mediadia=pd.DataFrame()
        for dia in fechas_dia:
            pattern=dia
            matching = [s for s in datos_estaciones.FECHA if pattern in s]
            fecha_inicio=matching[0]
            fecha_fin=matching[-1]
            media2020=mapa2020.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
            media2020=media2020.stack().reset_index()
            media2020['fecha']=pattern
            media2020['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
            media2020['contaminacion']=media2020['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
            media2020['contaminacion']=media2020['contaminacion'].astype(np.int32)
            media2020['fecha']=pd.to_datetime(media2020['fecha'])
            mapa2020_mediadia=mapa2020_mediadia.append(media2020)
        mapa2020_mediadia.to_csv('./data/mapa2017_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')
        
   # Se convierte el formato de matriz en 3D a una matriz 2D
    mapa_interp=mapa.stack().reset_index()
    mapa_interp.columns=['fecha','latitud','longitud','contaminacion']
    mapa_interp['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
    mapa_interp['contaminacion']=mapa_interp['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
    mapa_interp['contaminacion']=mapa_interp['contaminacion'].astype(np.int32)
    mapa_interp['fecha']=pd.to_datetime(mapa_interp['fecha'])
    filename='./data/mapa_interpolado'+ano
    filenamecsv='./data/mapa_interpolado'+ano+'.csv'
    mapa_interp.to_feather(filename)
    print('Guardado feather '+ano)
    mapa_interp.to_csv(filenamecsv,chunksize=200000,mode='a')

2016-01-01 00:00:00


/anaconda3/lib/python3.6/site-packages/scipy/interpolate/fitpack2.py:1057: UserWarning: 
The required storage space exceeds the available storage space: nxest
or nyest too small, or s too small.
The weighted least-squares spline corresponds to the current set of
knots.
  warnings.warn(message)


2016-01-01 01:00:00
2016-01-01 02:00:00
2016-01-01 03:00:00
2016-01-01 04:00:00
2016-01-01 05:00:00
2016-01-01 06:00:00
2016-01-01 07:00:00
2016-01-01 08:00:00
2016-01-01 09:00:00
2016-01-01 10:00:00
2016-01-01 11:00:00
2016-01-01 12:00:00
2016-01-01 13:00:00
2016-01-01 14:00:00
2016-01-01 15:00:00
2016-01-01 16:00:00
2016-01-01 17:00:00
2016-01-01 18:00:00
2016-01-01 19:00:00
2016-01-01 20:00:00
2016-01-01 21:00:00
2016-01-01 22:00:00
2016-01-01 23:00:00
2016-01-02 00:00:00
2016-01-02 01:00:00
2016-01-02 02:00:00
2016-01-02 03:00:00
2016-01-02 04:00:00
2016-01-02 05:00:00
2016-01-02 06:00:00
2016-01-02 07:00:00
2016-01-02 08:00:00
2016-01-02 09:00:00
2016-01-02 10:00:00
2016-01-02 11:00:00
2016-01-02 12:00:00
2016-01-02 13:00:00
2016-01-02 14:00:00
2016-01-02 15:00:00
2016-01-02 16:00:00
2016-01-02 17:00:00
2016-01-02 18:00:00
2016-01-02 19:00:00
2016-01-02 20:00:00
2016-01-02 21:00:00
2016-01-02 22:00:00
2016-01-02 23:00:00
2016-01-03 00:00:00
2016-01-03 01:00:00
2016-01-03 02:00:00


2016-01-18 04:00:00
2016-01-18 05:00:00
2016-01-18 06:00:00
2016-01-18 07:00:00
2016-01-18 08:00:00
2016-01-18 09:00:00
2016-01-18 10:00:00
2016-01-18 11:00:00
2016-01-18 12:00:00
2016-01-18 13:00:00
2016-01-18 14:00:00
2016-01-18 15:00:00
2016-01-18 16:00:00
2016-01-18 17:00:00
2016-01-18 18:00:00
2016-01-18 19:00:00
2016-01-18 20:00:00
2016-01-18 21:00:00
2016-01-18 22:00:00
2016-01-18 23:00:00
2016-01-19 00:00:00
2016-01-19 01:00:00
2016-01-19 02:00:00
2016-01-19 03:00:00
2016-01-19 04:00:00
2016-01-19 05:00:00
2016-01-19 06:00:00
2016-01-19 07:00:00
2016-01-19 08:00:00
2016-01-19 09:00:00
2016-01-19 10:00:00
2016-01-19 11:00:00
2016-01-19 12:00:00
2016-01-19 13:00:00
2016-01-19 14:00:00
2016-01-19 15:00:00
2016-01-19 16:00:00
2016-01-19 17:00:00
2016-01-19 18:00:00
2016-01-19 19:00:00
2016-01-19 20:00:00
2016-01-19 21:00:00
2016-01-19 22:00:00
2016-01-19 23:00:00
2016-01-20 00:00:00
2016-01-20 01:00:00
2016-01-20 02:00:00
2016-01-20 03:00:00
2016-01-20 04:00:00
2016-01-20 05:00:00


2016-02-04 07:00:00
2016-02-04 08:00:00
2016-02-04 09:00:00
2016-02-04 10:00:00
2016-02-04 11:00:00
2016-02-04 12:00:00
2016-02-04 13:00:00
2016-02-04 14:00:00
2016-02-04 15:00:00
2016-02-04 16:00:00
2016-02-04 17:00:00
2016-02-04 18:00:00
2016-02-04 19:00:00
2016-02-04 20:00:00
2016-02-04 21:00:00
2016-02-04 22:00:00
2016-02-04 23:00:00
2016-02-05 00:00:00
2016-02-05 01:00:00
2016-02-05 02:00:00
2016-02-05 03:00:00
2016-02-05 04:00:00
2016-02-05 05:00:00
2016-02-05 06:00:00
2016-02-05 07:00:00
2016-02-05 08:00:00
2016-02-05 09:00:00
2016-02-05 10:00:00
2016-02-05 11:00:00
2016-02-05 12:00:00
2016-02-05 13:00:00
2016-02-05 14:00:00
2016-02-05 15:00:00
2016-02-05 16:00:00
2016-02-05 17:00:00
2016-02-05 18:00:00
2016-02-05 19:00:00
2016-02-05 20:00:00
2016-02-05 21:00:00
2016-02-05 22:00:00
2016-02-05 23:00:00
2016-02-06 00:00:00
2016-02-06 01:00:00
2016-02-06 02:00:00
2016-02-06 03:00:00
2016-02-06 04:00:00
2016-02-06 05:00:00
2016-02-06 06:00:00
2016-02-06 07:00:00
2016-02-06 08:00:00


2016-02-21 10:00:00
2016-02-21 11:00:00
2016-02-21 12:00:00
2016-02-21 13:00:00
2016-02-21 14:00:00
2016-02-21 15:00:00
2016-02-21 16:00:00
2016-02-21 17:00:00
2016-02-21 18:00:00
2016-02-21 19:00:00
2016-02-21 20:00:00
2016-02-21 21:00:00
2016-02-21 22:00:00
2016-02-21 23:00:00
2016-02-22 00:00:00
2016-02-22 01:00:00
2016-02-22 02:00:00
2016-02-22 03:00:00
2016-02-22 04:00:00
2016-02-22 05:00:00
2016-02-22 06:00:00
2016-02-22 07:00:00
2016-02-22 08:00:00
2016-02-22 09:00:00
2016-02-22 10:00:00
2016-02-22 11:00:00
2016-02-22 12:00:00
2016-02-22 13:00:00
2016-02-22 14:00:00
2016-02-22 15:00:00
2016-02-22 16:00:00
2016-02-22 17:00:00
2016-02-22 18:00:00
2016-02-22 19:00:00
2016-02-22 20:00:00
2016-02-22 21:00:00
2016-02-22 22:00:00
2016-02-22 23:00:00
2016-02-23 00:00:00
2016-02-23 01:00:00
2016-02-23 02:00:00
2016-02-23 03:00:00
2016-02-23 04:00:00
2016-02-23 05:00:00
2016-02-23 06:00:00
2016-02-23 07:00:00
2016-02-23 08:00:00
2016-02-23 10:00:00
2016-02-23 11:00:00
2016-02-23 12:00:00


2016-03-09 13:00:00
2016-03-09 14:00:00
2016-03-09 16:00:00
2016-03-09 17:00:00
2016-03-09 18:00:00
2016-03-09 19:00:00
2016-03-09 20:00:00
2016-03-09 21:00:00
2016-03-09 22:00:00
2016-03-09 23:00:00
2016-03-10 00:00:00
2016-03-10 01:00:00
2016-03-10 02:00:00
2016-03-10 03:00:00
2016-03-10 04:00:00
2016-03-10 05:00:00
2016-03-10 06:00:00
2016-03-10 07:00:00
2016-03-10 08:00:00
2016-03-10 09:00:00
2016-03-10 10:00:00
2016-03-10 11:00:00
2016-03-10 12:00:00
2016-03-10 13:00:00
2016-03-10 14:00:00
2016-03-10 15:00:00
2016-03-10 16:00:00
2016-03-10 17:00:00
2016-03-10 18:00:00
2016-03-10 19:00:00
2016-03-10 20:00:00
2016-03-10 21:00:00
2016-03-10 22:00:00
2016-03-10 23:00:00
2016-03-11 00:00:00
2016-03-11 01:00:00
2016-03-11 02:00:00
2016-03-11 03:00:00
2016-03-11 04:00:00
2016-03-11 05:00:00
2016-03-11 06:00:00
2016-03-11 07:00:00
2016-03-11 08:00:00
2016-03-11 09:00:00
2016-03-11 10:00:00
2016-03-11 11:00:00
2016-03-11 12:00:00
2016-03-11 13:00:00
2016-03-11 14:00:00
2016-03-11 15:00:00


2016-03-26 17:00:00
2016-03-26 18:00:00
2016-03-26 19:00:00
2016-03-26 20:00:00
2016-03-26 21:00:00
2016-03-26 22:00:00
2016-03-26 23:00:00
2016-03-27 00:00:00
2016-03-27 01:00:00
2016-03-27 02:00:00
2016-03-27 03:00:00
2016-03-27 04:00:00
2016-03-27 05:00:00
2016-03-27 06:00:00
2016-03-27 07:00:00
2016-03-27 08:00:00
2016-03-27 09:00:00
2016-03-27 10:00:00
2016-03-27 11:00:00
2016-03-27 12:00:00
2016-03-27 13:00:00
2016-03-27 14:00:00
2016-03-27 15:00:00
2016-03-27 16:00:00
2016-03-27 17:00:00
2016-03-27 18:00:00
2016-03-27 19:00:00
2016-03-27 20:00:00
2016-03-27 21:00:00
2016-03-27 22:00:00
2016-03-27 23:00:00
2016-03-28 00:00:00
2016-03-28 01:00:00
2016-03-28 02:00:00
2016-03-28 03:00:00
2016-03-28 04:00:00
2016-03-28 05:00:00
2016-03-28 06:00:00
2016-03-28 07:00:00
2016-03-28 08:00:00
2016-03-28 09:00:00
2016-03-28 10:00:00
2016-03-28 11:00:00
2016-03-28 12:00:00
2016-03-28 13:00:00
2016-03-28 14:00:00
2016-03-28 15:00:00
2016-03-28 16:00:00
2016-03-28 17:00:00
2016-03-28 18:00:00


In [ ]:
print(ano)
#se coge mapa
mapa2016_mediames=pd.DataFrame()
for dia in fechas_dia:
    pattern=dia
    matching = [s for s in datos_estaciones.FECHA if pattern in s]
    fecha_inicio=matching[0]
    fecha_fin=matching[-1]
    media2016=mapa2016.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
    media2016=media2016.stack()
    media2016.reset_index(inplace=True)
    media2016['fecha']=pattern
    media2016['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
    media2016['contaminacion']=media2016['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
    media2016['contaminacion']=media2016['contaminacion'].astype(np.int32)
    media2016['fecha']=pd.to_datetime(media2016['fecha'])
    mapa2016_mediadia=mapa2016_mediadia.append(media2016)
mapa2016_mediadia.to_csv('./data/mapa2016_mediames.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')

In [ ]:
if ano=='2016':
    mapa2016_mediadia=pd.DataFrame()
    for dia in fechas_dia:
        pattern=dia
        matching = [s for s in datos_estaciones.FECHA if pattern in s]
        fecha_inicio=matching[0]
        fecha_fin=matching[-1]
        media2016=mapa2016.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
        media2016=media2016.stack().reset_index()
        media2016['fecha']=pattern
        media2016['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
        media2016['contaminacion']=media2016['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
        media2016['contaminacion']=media2016['contaminacion'].astype(np.int32)
        media2016['fecha']=pd.to_datetime(media2016['fecha'])
        mapa2016_mediadia=mapa2016_mediadia.append(media2016)
    mapa2016_mediadia.to_csv('./data/mapa2016_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')

elif ano=='2017':
    mapa2017_mediadia=pd.DataFrame()
    for dia in fechas_dia:
        pattern=dia
        matching = [s for s in datos_estaciones.FECHA if pattern in s]
        fecha_inicio=matching[0]
        fecha_fin=matching[-1]
        media2017=mapa2017.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
        media2017=media2017.stack().reset_index()
        media2017['fecha']=pattern
        media2017['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
        media2017['contaminacion']=media2017['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
        media2017['contaminacion']=media2017['contaminacion'].astype(np.int32)
        media2017['fecha']=pd.to_datetime(media2017['fecha'])
        mapa2017_mediadia=mapa2017_mediadia.append(media2017)
    mapa2017_mediadia.to_csv('./data/mapa2017_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')

elif ano=='2018':
    mapa2018_mediadia=pd.DataFrame()
    for dia in fechas_dia:
        pattern=dia
        matching = [s for s in datos_estaciones.FECHA if pattern in s]
        fecha_inicio=matching[0]
        fecha_fin=matching[-1]
        media2018=mapa2018.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
        media2018=media2018.stack().reset_index()
        media2018['fecha']=pattern
        media2018['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
        media2018['contaminacion']=media2018['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
        media2018['contaminacion']=media2018['contaminacion'].astype(np.int32)
        media2018['fecha']=pd.to_datetime(media2018['fecha'])
        mapa2018_mediadia=mapa2018_mediadia.append(media2017)
    mapa2018_mediadia.to_csv('./data/mapa2018_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')

elif ano=='2019':
    mapa2019_mediadia=pd.DataFrame()
    for dia in fechas_dia:
        pattern=dia
        matching = [s for s in datos_estaciones.FECHA if pattern in s]
        fecha_inicio=matching[0]
        fecha_fin=matching[-1]
        media2019=mapa2019.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
        media2019=media2019.stack().reset_index()
        media2019['fecha']=pattern
        media2019['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
        media2019['contaminacion']=media2019['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
        media2019['contaminacion']=media2019['contaminacion'].astype(np.int32)
        media2019['fecha']=pd.to_datetime(media2019['fecha'])
        mapa2019_mediadia=mapa2019_mediadia.append(media2019)
    mapa2019_mediadia.to_csv('./data/mapa2019_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')

else:
    mapa2020_mediadia=pd.DataFrame()
    for dia in fechas_dia:
        pattern=dia
        matching = [s for s in datos_estaciones.FECHA if pattern in s]
        fecha_inicio=matching[0]
        fecha_fin=matching[-1]
        media2020=mapa2020.query("fecha >= @fecha_inicio & fecha <= @fecha_fin").mean(level='latitud')
        media2020=media2020.stack().reset_index()
        media2020['fecha']=pattern
        media2020['contaminacion'].clip(lower=0, upper=None, axis=None, inplace=True)
        media2020['contaminacion']=media2020['contaminacion'].apply(lambda v: 1 if v<=40 else 2 if ((v>=41)&(v<=100)) else 3 if ((v>=101)&(v<=200)) else 4 if ((v>=201)&(v<=400)) else 5)
        media2020['contaminacion']=media2020['contaminacion'].astype(np.int32)
        media2020['fecha']=pd.to_datetime(media2020['fecha'])
        mapa2020_mediadia=mapa2020_mediadia.append(media2020)
    mapa2020_mediadia.to_csv('./data/mapa2017_mediadia.csv',columns=['latitud','longitud','contaminacion','fecha'],sep=',',encoding='utf-8')
